# Introduction to Protocol Buffers (Protobuf)

Protocol Buffers (Protobuf) is a language-neutral, platform-neutral, extensible mechanism for serializing structured data. Developed by Google, Protobuf is used to define data structures and services, and to serialize data efficiently for communication between programs.

**Key Features:**
- Compact and fast serialization format.
- Strongly typed schema definition.
- Supports code generation for multiple languages (e.g., Go, Python, Java).

**Example `.proto` file:**

In [ ]:
syntax = "proto3";

package example;

// Message definition
message Person {
  string name = 1;
  int32 id = 2;
  string email = 3;
}

# How Protobuf Works: From .proto to Code

The typical workflow for using Protobuf involves several steps:

1. **Define your data structure in a `.proto` file.**
2. **Run the Protocol Buffer Compiler (`protoc`)** to generate code in your target language.
3. **Use the generated code** in your application to create, serialize, and parse messages.

Below is a diagram illustrating this process:

```
+----------------+         protoc         +---------------------+         +--------------------------+
|  person.proto  |  ------------------>   |  Generated Library  |  --->   |  Your Application Code   |
+----------------+   (protoc --go_out)    +---------------------+         +--------------------------+
        |                                         |                                 |
        |  message Person { ... }                 |  // Person struct/class         |
        |                                         |  // Getters/Setters             |
        |                                         |  // Serialization methods       |
```

### Example: Using Generated Code (Go)

```go
person := &example.Person{}
person.Name = "Alice"           // Setter
person.Id = 123                  // Setter
fmt.Println(person.GetName())     // Getter

// Serialize to bytes
data, err := proto.Marshal(person)

// Parse from bytes
newPerson := &example.Person{}
err := proto.Unmarshal(data, newPerson)
```

The generated code provides type-safe accessors (getters/setters) and serialization methods for your data.

# Introduction to gRPC

gRPC is a high-performance, open-source universal RPC (Remote Procedure Call) framework developed by Google. It enables communication between distributed services using HTTP/2 for transport, Protocol Buffers for interface definition, and provides features such as authentication, load balancing, and more.

**How gRPC uses Protobuf:**
- Service contracts (APIs) are defined in `.proto` files.
- Protobuf messages define the data structures exchanged.
- Code generation tools create client and server stubs in various languages.

**Typical gRPC workflow:**
1. Define service and messages in a `.proto` file.
2. Generate code for server and client.
3. Implement server logic and call services from clients.

# Simple Example of gRPC

Below is a basic example of a gRPC service definition in Protobuf, followed by a demonstration of how to implement a server and client in Go.

**Service Definition (`helloworld.proto`):**

In [ ]:
syntax = "proto3";

package helloworld;

// The greeting service definition.
service Greeter {
  // Sends a greeting
  rpc SayHello (HelloRequest) returns (HelloReply) {}
}

// The request message containing the user's name.
message HelloRequest {
  string name = 1;
}

// The response message containing the greetings.
message HelloReply {
  string message = 1;
}

# Sample Go Code for gRPC

Below is a simplified Go implementation of a gRPC server and client for the `Greeter` service.

**Server Implementation:**

In [ ]:
package main

import (
    "context"
    "log"
    "net"

    "google.golang.org/grpc"
    pb "path/to/helloworld"
)

type server struct {
    pb.UnimplementedGreeterServer
}

func (s *server) SayHello(ctx context.Context, in *pb.HelloRequest) (*pb.HelloReply, error) {
    return &pb.HelloReply{Message: "Hello " + in.Name}, nil
}

func main() {
    lis, err := net.Listen("tcp", ":50051")
    if err != nil {
        log.Fatalf("failed to listen: %v", err)
    }
    s := grpc.NewServer()
    pb.RegisterGreeterServer(s, &server{})
    log.Println("Server listening at :50051")
    if err := s.Serve(lis); err != nil {
        log.Fatalf("failed to serve: %v", err)
    }
}

**Client Implementation:**

In [ ]:
package main

import (
    "context"
    "log"
    "os"
    "time"

    "google.golang.org/grpc"
    pb "path/to/helloworld"
)

func main() {
    conn, err := grpc.Dial("localhost:50051", grpc.WithInsecure())
    if err != nil {
        log.Fatalf("did not connect: %v", err)
    }
    defer conn.Close()
    c := pb.NewGreeterClient(conn)

    name := "world"
    if len(os.Args) > 1 {
        name = os.Args[1]
    }
    ctx, cancel := context.WithTimeout(context.Background(), time.Second)
    defer cancel()
    r, err := c.SayHello(ctx, &pb.HelloRequest{Name: name})
    if err != nil {
        log.Fatalf("could not greet: %v", err)
    }
    log.Printf("Greeting: %s", r.Message)
}

# What is gNOI?

gNOI (gRPC Network Operations Interface) is a set of gRPC-based APIs designed for network device management and operations. It extends gRPC and Protobuf to provide standardized interfaces for common operational tasks on network devices, such as software installation, certificate management, and system reboot.

**Repository:** [https://github.com/openconfig/gnoi](https://github.com/openconfig/gnoi)

**Key Points:**
- gNOI defines a set of services for operational tasks.
- Enables automation and consistency across network devices.
- Uses Protobuf for message definitions and gRPC for transport.

**Common gNOI Services:**
- System (e.g., Time, Reboot)
- Certificate Management
- OS Installation

# Simple Examples of gNOI

Below are examples of using gNOI services in Go, such as querying system time and rebooting a device.

**Example: System.Time Request**

In [ ]:
import (
    "context"
    "log"
    "time"

    "google.golang.org/grpc"
    gnoi_system "github.com/openconfig/gnoi/system"
)

func getSystemTime(address string) {
    conn, err := grpc.Dial(address, grpc.WithInsecure())
    if err != nil {
        log.Fatalf("failed to connect: %v", err)
    }
    defer conn.Close()

    client := gnoi_system.NewSystemClient(conn)
    ctx, cancel := context.WithTimeout(context.Background(), 5*time.Second)
    defer cancel()

    resp, err := client.Time(ctx, &gnoi_system.TimeRequest{})
    if err != nil {
        log.Fatalf("failed to get time: %v", err)
    }
    log.Printf("System time: %v", resp.GetTime())
}

**Example: System.Reboot Request**

In [ ]:
import (
    "context"
    "log"
    "time"

    "google.golang.org/grpc"
    gnoi_system "github.com/openconfig/gnoi/system"
)

func rebootDevice(address string) {
    conn, err := grpc.Dial(address, grpc.WithInsecure())
    if err != nil {
        log.Fatalf("failed to connect: %v", err)
    }
    defer conn.Close()

    client := gnoi_system.NewSystemClient(conn)
    ctx, cancel := context.WithTimeout(context.Background(), 5*time.Second)
    defer cancel()

    _, err = client.Reboot(ctx, &gnoi_system.RebootRequest{Method: gnoi_system.RebootMethod_COLD})
    if err != nil {
        log.Fatalf("failed to reboot: %v", err)
    }
    log.Println("Reboot command sent successfully")
}

# Step-by-Step Guide: Trying Out gNOI

Follow these steps to try out gNOI in a virtual testbed environment:

0. **Set up your VS testbed** ([instructions](https://github.com/sonic-net/sonic-mgmt/blob/75bdedd7e842a54052d0889c4450bcf88b7dcd55/docs/testbed/README.testbed.VsSetup.md))

1. **Install gnoic (gnoi client) on azure vm (with VS set up):**
   ```bash
   bash -c "$(curl -sL https://get-gnoic.kmrd.dev)"
   ```

2. **Check the gNOI server process on your device:**
   ```bash
   admin@vlab-01:~$ docker exec gnmi ps -efwww
   UID          PID    PPID  C STIME TTY          TIME CMD
   root           1       0  0 Apr25 pts/0    00:00:51 /usr/bin/python3 /usr/local/bin/supervisord
   root           8       1  0 Apr25 pts/0    00:00:21 python3 /usr/bin/supervisor-proc-exit-listener --container-name gnmi
   root          11       1  0 Apr25 pts/0    00:00:00 /usr/sbin/rsyslogd -n -iNONE
   root          18       1  0 Apr25 pts/0    00:16:32 /usr/sbin/telemetry -logtostderr --noTLS --port 8080 --allow_no_client_auth -v=2 --threshold 100 --idle_conn_duration 5
   root          61       1  0 Apr25 pts/0    00:16:32 /usr/sbin/dialout_client_cli -insecure -logtostderr -v 2
   root          71       0 50 21:18 ?        00:00:00 ps -efwww
   ```

3. **Run the gNOI time command:**
   ```bash
   gnoic system time --address 10.250.0.101 --port 8080 --insecure
   ```
   Example output:
   |    Target Name    |                  Time                   |      Timestamp      |
   |-------------------|-----------------------------------------|---------------------|
   | 10.250.0.101:8080 | 2025-04-28 21:20:33.305614793 +0000 UTC | 1745875233305614793 |

You have now successfully queried the system time using gNOI!